# **Task 4: General Health Query Chatbot (Prompt Engineering)**

# **Organization: DevelopersHub Corporation**
# **Author:** Hifza Nazir

# **Date:** 11 February 2026

# **Step 1: Install & Import Required Libraries**

In [ ]:
# Install necessary libraries for model loading and processing
!pip install transformers accelerate torch bitsandbytes -q
print("✅ Libraries installed!")

# Import required modules
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import zipfile
import os
import shutil
from google.colab import files
print("✅ Libraries imported!")

✅ Libraries installed!
✅ Libraries imported!


# **Step 2: Load the AI Model**

In [ ]:
# Install transformers, accelerate, and torch if not already installed
!pip install transformers accelerate torch -q

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

print("Loading model...")
# Load the TinyLlama-1.1B-Chat-v1.0 model from Hugging Face
# torch_dtype=torch.float16 uses half-precision floats for memory efficiency
# device_map="auto" automatically distributes the model layers across available devices (e.g., GPU)
model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.float16,
    device_map="auto"
)
# Load the tokenizer for the TinyLlama model
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
print("Model loaded!")

Loading model...


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Model loaded!


# **Step 3: Implement Prompt Engineering (The Chatbot Logic)**

In [ ]:
def health_chatbot(user_query):
    """
    Health Assistant Chatbot with Prompt Engineering
    """

    # PROMPT ENGINEERING: Define AI behavior and rules for the chatbot
    system_prompt = (
        "<|system|>\n"
        "You are a helpful, friendly, and empathetic Medical Assistant. "
        "Your name is 'HealthMate AI'. "
        "\n\n"
        "IMPORTANT RULES:\n"
        "1. Always start with a friendly greeting\n"
        "2. Provide clear, science-based information about general health\n"
        "3. Use simple, easy-to-understand language\n"
        "4. MUST include this disclaimer: "
        "' DISCLAIMER: I am an AI assistant, not a doctor. This information is for educational purposes only. "
        "Please consult a healthcare professional for medical advice.'\n"
        "5. If user mentions emergency symptoms, immediately advise to call ambulance\n"
        "6. Never prescribe medications or give specific dosages\n"
        "7. Be empathetic and kind\n"
    )

    # SAFETY FILTERS: Keywords to detect potential medical emergencies
    emergency_keywords = [
        "emergency", "ambulance", "heart attack", "stroke", "severe bleeding",
        "not breathing", "unconscious", "dying", "suicide", "chest pain",
        "difficulty breathing", "severe injury"
    ]

    # Check if the user's query contains any emergency keywords
    is_emergency = any(keyword in user_query.lower() for keyword in emergency_keywords)

    # Create the full prompt by combining the system prompt, user query, and an assistant marker
    if is_emergency:
        # Append an emergency alert to the system prompt if an emergency is detected
        emergency_alert = (
            "\n EMERGENCY DETECTED: This appears to be a medical emergency! "
            "Prioritize getting professional help immediately.\n"
        )
        system_prompt += emergency_alert

    # Format the full prompt for the model
    full_prompt = f"{system_prompt}<|user|>\n{user_query}<|assistant|>\n"

    # Generate response from the model
    try:
        # Use the 'pipe' object (assuming it's defined globally or passed) to generate text
        # max_new_tokens: maximum number of tokens to generate
        # do_sample: use sampling for generation
        # temperature: controls randomness (lower = less random)
        # top_k: limits the vocabulary to the top_k most likely tokens
        # top_p: filters tokens based on cumulative probability
        outputs = pipe(
            full_prompt,
            max_new_tokens=300,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95
        )

        # Extract the assistant's response from the generated text
        response = outputs[0]["generated_text"].split("<|assistant|>\n")[-1]

        # Prepend an emergency header if an emergency was detected
        if is_emergency:
            response = " **MEDICAL EMERGENCY DETECTED** \n\n**IMMEDIATE ACTION REQUIRED:** Please call emergency services (911) right away!\n\n---\n\n" + response

        return response

    except Exception as e:
        # Handle any errors during response generation
        return f"I apologize, but I encountered an error. Please try again. Error: {str(e)}"

print(" Health Assistant function created with Prompt Engineering!")

 Health Assistant function created with Prompt Engineering!


# **Step 4: Testing the Chatbot**

In [ ]:
# Test Query 1: Asking about common health symptoms
print("="*60)
print(" TEST QUERY 1: What causes a sore throat?")
print("="*60)
query1 = "What causes a sore throat?"
response1 = health_chatbot(query1)
print(f" User: {query1}")
print(f" HealthMate AI: {response1}")
print("\n" + "="*60)

# Test Query 2: Asking about medication safety for a specific demographic
print(" TEST QUERY 2: Is paracetamol safe for children?")
print("="*60)
query2 = "Is paracetamol safe for children?"
response2 = health_chatbot(query2)
print(f" User: {query2}")
print(f" HealthMate AI: {response2}")
print("\n" + "="*60)

# Test Query 3: Posing an emergency scenario to test safety filters
print(" TEST QUERY 3: I have severe chest pain")
print("="*60)
query3 = "I have severe chest pain, what should I do?"
response3 = health_chatbot(query3)
print(f" User: {query3}")
print(f" HealthMate AI: {response3}")
print("\n" + "="*60)

print(" All tests completed!")

 TEST QUERY 1: What causes a sore throat?
 User: What causes a sore throat?
 HealthMate AI: I apologize, but I encountered an error. Please try again. Error: name 'pipe' is not defined

 TEST QUERY 2: Is paracetamol safe for children?
 User: Is paracetamol safe for children?
 HealthMate AI: I apologize, but I encountered an error. Please try again. Error: name 'pipe' is not defined

 TEST QUERY 3: I have severe chest pain
 User: I have severe chest pain, what should I do?
 HealthMate AI: I apologize, but I encountered an error. Please try again. Error: name 'pipe' is not defined

 All tests completed!


# **Step 5: Save Model**

In [ ]:
from google.colab import drive
import os

# Mount Google Drive to access files
drive.mount('/content/drive')

# Set the path in Google Drive where the model will be saved
model_save_path = "/content/drive/MyDrive/tinyllama_model"

# Check if the model directory already exists in Drive
if not os.path.exists(model_save_path):
    print("Saving model to Drive for the first time...")
    # Save the model and tokenizer to the specified path
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)
    print("Model saved to Drive!")
else:
    print("Model already exists in Drive. Loading from Drive (Fast)...")
    # In a full implementation, you would load the model here:
    # model = AutoModelForCausalLM.from_pretrained(model_save_path)
    # tokenizer = AutoTokenizer.from_pretrained(model_save_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model already exists in Drive. Loading from Drive (Fast)...


# **Summary:**

# **1. Objective**
The goal of this task was to develop an AI-powered Health Assistant capable of answering general medical queries using a Large Language Model (LLM). The focus was on applying **Prompt Engineering** techniques to control the AI's behavior, tone, and safety protocols.

# **2. Methodology & Tools**
# Model:
We utilized the **TinyLlama-1.1B-Chat**, an open-source Large Language Model hosted on Hugging Face.

# Environment:
The project was implemented in **Google Colab** using the transformers and accelerate libraries.

# Inference Strategy:
Instead of using paid external APIs (like OpenAI or Gemini), we loaded the model locally on the Colab GPU. This approach was chosen to understand the deployment of open-source models without the need for API keys or subscription costs.

# **3. Prompt Engineering & Safety**
The core of this task was "programming" the AI through a **System Prompt**. We engineered the instructions to ensure the chatbot:

Acts as a **professional and empathetic** medical assistant.

Provides **science-based** information.

Includes a mandatory **medical disclaimer** ("I am an AI, not a doctor").

Prioritizes **emergency safety** by directing users to professional medical help for urgent symptoms.

# **4. Key Results**
The chatbot successfully handled various health-related queries, such as "What causes a sore throat?" and "Is paracetamol safe for kids?". It demonstrated the ability to provide informative answers while strictly adhering to the safety constraints defined in the system instructions.

# **5. Conclusion**
This task demonstrated that with effective Prompt Engineering, even smaller open-source models can be transformed into specialized, safe, and helpful assistants for specific domains like healthcare.